In [1]:
import fileio
import pydata

#import numpy as np
#import pandas as pd

In [12]:
import shapes

In [ ]:
from importlib import reload

In [ ]:
reload(fileio)
#reload(shapes)

In [2]:
myfile = fileio.InputData('arecaceae.csv')
myfile.iucnFile('arecaceae_categories.csv')

In [ ]:
myar = shapes.KBA('/home/nelson/Dropbox/Humboldt/Postdoc/KBA_by_IUCN/Colombia_KBA', 'SitRecID')

In [ ]:
myar.spp_inclusion(myfile)

In [ ]:
myar.new_spp_table('test_kba_log.csv')

---

In [6]:
tilas = myfile.getTiles(0.1, offsetLat=0, offsetLon=0)

In [ ]:
for ti in tilas:
    print(myfile.tile2str(ti))

In [15]:
mysols = pydata.metasearchAlt(tilas, 0.2, 10000, 30, 50)

In [16]:
for sol in mysols[0]:
    print(sol.origin)

0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
1
0
0
0
0
0
1
0
0
0
0


In [ ]:
mysols[0][0].aggrScore, mysols[0][0].score, mysols[0][0].ndmScore

In [17]:
rm -r solutions

In [18]:
shapes.solution2shape(mysols, myfile)

In [ ]:
procAreas = {}

filehandle = fiona.open('/home/nelson/Data/GIS/Areas_protegidas/RUNAP/runap2Polygon.shp', crs= 'EPSG:4326', encoding = 'utf8')
for item in filehandle:
    #self.polys.append(shape(item['geometry']))
    procAreas[item['properties']['id_pnn']] = {
        'shape': shape(item['geometry'])
        }

In [ ]:
procAreas['20170004']['shape'].contains

In [ ]:
polys = []
for ic in range(mysols[1][0].getSize()):
    if mysols[1][0].getValue(ic) > 0:
        y, x = irkeys[ic]
        xBase = myfile.originN[0] + myfile.cellSize * x
        yBase = myfile.originN[1] - myfile.cellSize * y
        ocor = [(xBase + myfile.cellSize, yBase),
            (xBase, yBase),
            (xBase, yBase - myfile.cellSize),
            (xBase + myfile.cellSize, yBase - myfile.cellSize)]
        polys.append(Polygon(ocor))
solpoly = unary_union(polys)

for sp in myfile.points:
    for lon, lat in myfile.points[sp]:
        if solpoly.contains(Point(lon, lat)):
            print (sp)


In [ ]:
solpoly.contains

In [ ]:
schema = {
    'geometry': 'Polygon',
    'properties': {'id': 'int',
                  'score': 'float',
                  'NDMscore': 'float'},
    }

irkeys = list(myfile.index_reg.keys())

for igr, gr in enumerate(mysols):
    solpolcoll = []
    multipol = None
    filename = 'group_{0}.shp'.format(igr)

    for its,  tsol in enumerate(gr):
        polys = []
        solpoly = None
        for ic in range(tsol.getSize()):
            if tsol.getValue(ic) > 0:
                y, x = irkeys[ic]
                xBase = myfile.originN[0] + myfile.cellSize * x
                yBase = myfile.originN[1] - myfile.cellSize * y
                ocor = [(xBase + myfile.cellSize, yBase),
                    (xBase, yBase),
                    (xBase, yBase - myfile.cellSize),
                    (xBase + myfile.cellSize, yBase - myfile.cellSize)]
                polys.append(Polygon(ocor))
        solpoly = unary_union(polys)
        
        if its == 0:
                with fiona.open(filename, 'w', 'ESRI Shapefile', schema, from_epsg(4326)) as c:
                    c.write({'geometry': mapping(solpoly),
                        'properties': {'id': ip,
                                      'score':  tsol.aggrScore,
                                      'NDMscore': tsol.ndmScore}
                            })

        else:
            with fiona.open(filename, 'a', 'ESRI Shapefile', schema, from_epsg(4326)) as c:
                c.write({'geometry': mapping(solpoly),
                    'properties': {'id': ip,
                                      'score':  tsol.aggrScore,
                                      'NDMscore': tsol.ndmScore}
                        })


In [ ]:
for spp in [tilas[x].getName() for x in mysols[0][0].spp2crit.keys()]:
    print(spp, len(myfile.points[spp]))

In [ ]:
print(myfile.tile2str( mysols[1][0]))

In [ ]:
mysols = pydata.metasearchAlt(tilas, 0.2, 1000, 500, 2)

for ig, group in enumerate(mysols):
    print ('#' * 50)
    print ('Group {0}'.format(ig))
    for iso, so in enumerate(group):
        print("Solution {0} of {1}".format(iso, len(group)))
        print(so.score, "-", so.ndmScore, "-", so.aggrScore)
        #print("Islands: ", pydata.islNum(so))
        #if not pydata.isCont(so):
        #    print("NOT CONTINUOUS!!!")
        #print(so.toBitList())
        print(myfile.tile2str(so))
    